### Request data

In [1]:
import requests
import pandas as pd
import json
import ijson
pd.set_option('display.max_columns', None)
opendota_api_heros_url = 'https://api.opendota.com/api/heroes'

data = requests.get(opendota_api_heros_url, timeout=3)
data_json = data.json()

en_hero_df = pd.DataFrame(data_json)
en_hero_df['en_name'] = en_hero_df['name'].apply(lambda x:x.replace('npc_dota_hero_', ''))
en_hero_df = en_hero_df.rename(columns = {'localized_name': 'official_name', 'id': 'hero_id',\
                                          'name': 'ingame_name'})
en_hero_df.head()

,hero_id,ingame_name,official_name,primary_attr,attack_type,roles,legs,en_name
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",2,antimage
1,2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Carry]",2,axe
2,3,npc_dota_hero_bane,Bane,all,Ranged,"[Support, Disabler, Nuker, Durable]",4,bane
3,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Nuker, Initiator]",2,bloodseeker
4,5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker]",2,crystal_maiden


In [3]:
en_hero_df.to_csv('dota2_heroes.csv', index=False, header=True)

In [2]:
def get_api_json(url, loop=True, proxy=None):
    try:
        return requests.get(url, headers={'User-Agent': 'Chrome'}, timeout=3, proxies=proxy).json()
    except requests.exceptions.RequestException as e:
        print(e)
        return get_api_json(url, loop, proxy) if loop is True else None

In [4]:
get_api_json("http://api.opendota.com/api/players/148351321/wl", loop=True)

{'win': 769, 'lose': 842}

Select data

In [3]:
# data from "that" match
match_json = get_api_json("https://api.opendota.com/api/matches/6227419633", loop=True)

In [4]:
match_json['players'][0]['name']

'萧瑟'

In [5]:
hero_id_to_name = dict(zip(en_hero_df['hero_id'].values, en_hero_df['official_name'].values))
# hero_id_to_name

Using API to get the matches from 2023-01-01 to 2023-05-08

https://www.opendota.com/explorer

In [6]:
# SELECT
        
# matches.match_id,
# matches.start_time

# FROM matches
# JOIN match_patch using(match_id)

# WHERE matches.start_time >= extract(epoch from timestamp '2021-01-01T07:00:00.000Z')
# AND matches.start_time <= extract(epoch from timestamp '2023-05-09T07:00:00.000Z')
# ORDER BY matches.start_time DESC
# LIMIT 100000

match_id_api = 'https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%0A%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0A%0AWHERE%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272021-01-01T07%3A00%3A00.000Z%27)%0AAND%20matches.start_time%20%3C%3D%20extract(epoch%20from%20timestamp%20%272023-05-09T07%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.start_time%20DESC%0ALIMIT%20100000'
match_data = requests.get(match_id_api).json()

In [47]:
# new version data
# teams, GG-9panda

# SELECT
# matches.match_id,
# matches.start_time,
# ((player_matches.player_slot < 128) = matches.radiant_win) win,
# player_matches.hero_id,
# player_matches.account_id,
# leagues.name leaguename
# FROM matches
# JOIN match_patch using(match_id)
# JOIN leagues using(leagueid)
# JOIN player_matches using(match_id)
# JOIN heroes on heroes.id = player_matches.hero_id
# LEFT JOIN notable_players ON notable_players.account_id = player_matches.account_id
# LEFT JOIN teams using(team_id)
# WHERE TRUE
# AND (player_matches.account_id = 93552791 OR player_matches.account_id = 132851371 OR player_matches.account_id = 134556694 OR player_matches.account_id = 858106446 OR player_matches.account_id = 161839895 OR player_matches.account_id = 165564598 OR player_matches.account_id = 930586713 OR player_matches.account_id = 292921272 OR player_matches.account_id = 117089283 OR player_matches.account_id = 339235645 OR player_matches.account_id = 153836240 OR player_matches.account_id = 155332459 OR player_matches.account_id = 103735745 OR player_matches.account_id = 10366616 OR player_matches.account_id = 86698277 OR player_matches.account_id = 94786276 OR player_matches.account_id = 100058342 OR player_matches.account_id = 201358612 OR player_matches.account_id = 73562326 OR player_matches.account_id = 77490514 OR player_matches.account_id = 152962063 OR player_matches.account_id = 54580962 OR player_matches.account_id = 221666230 OR player_matches.account_id = 116934015 OR player_matches.account_id = 16497807 OR player_matches.account_id = 97590558 OR player_matches.account_id = 91730177)
# AND matches.start_time >= extract(epoch from timestamp '2023-04-21T07:00:00.000Z')
# ORDER BY matches.match_id NULLS LAST
# LIMIT 2000

match_id_api1='https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%2C%0A((player_matches.player_slot%20%3C%20128)%20%3D%20matches.radiant_win)%20win%2C%0Aplayer_matches.hero_id%2C%0Aplayer_matches.account_id%2C%0Aleagues.name%20leaguename%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0AJOIN%20leagues%20using(leagueid)%0AJOIN%20player_matches%20using(match_id)%0AJOIN%20heroes%20on%20heroes.id%20%3D%20player_matches.hero_id%0ALEFT%20JOIN%20notable_players%20ON%20notable_players.account_id%20%3D%20player_matches.account_id%0ALEFT%20JOIN%20teams%20using(team_id)%0AWHERE%20TRUE%0AAND%20(player_matches.account_id%20%3D%2093552791%20OR%20player_matches.account_id%20%3D%20132851371%20OR%20player_matches.account_id%20%3D%20134556694%20OR%20player_matches.account_id%20%3D%20858106446%20OR%20player_matches.account_id%20%3D%20161839895%20OR%20player_matches.account_id%20%3D%20165564598%20OR%20player_matches.account_id%20%3D%20930586713%20OR%20player_matches.account_id%20%3D%20292921272%20OR%20player_matches.account_id%20%3D%20117089283%20OR%20player_matches.account_id%20%3D%20339235645%20OR%20player_matches.account_id%20%3D%20153836240%20OR%20player_matches.account_id%20%3D%20155332459%20OR%20player_matches.account_id%20%3D%20103735745%20OR%20player_matches.account_id%20%3D%2010366616%20OR%20player_matches.account_id%20%3D%2086698277%20OR%20player_matches.account_id%20%3D%2094786276%20OR%20player_matches.account_id%20%3D%20100058342%20OR%20player_matches.account_id%20%3D%20201358612%20OR%20player_matches.account_id%20%3D%2073562326%20OR%20player_matches.account_id%20%3D%2077490514%20OR%20player_matches.account_id%20%3D%20152962063%20OR%20player_matches.account_id%20%3D%2054580962%20OR%20player_matches.account_id%20%3D%20221666230%20OR%20player_matches.account_id%20%3D%20116934015%20OR%20player_matches.account_id%20%3D%2016497807%20OR%20player_matches.account_id%20%3D%2097590558%20OR%20player_matches.account_id%20%3D%2091730177)%0AAND%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272023-04-21T07%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.match_id%20NULLS%20LAST%0ALIMIT%202000'

In [55]:
# 6-10
# SELECT
# matches.match_id,
# matches.start_time,
# ((player_matches.player_slot < 128) = matches.radiant_win) win,
# player_matches.hero_id,
# player_matches.account_id,
# leagues.name leaguename
# FROM matches
# JOIN match_patch using(match_id)
# JOIN leagues using(leagueid)
# JOIN player_matches using(match_id)
# JOIN heroes on heroes.id = player_matches.hero_id
# LEFT JOIN notable_players ON notable_players.account_id = player_matches.account_id
# LEFT JOIN teams using(team_id)
# WHERE TRUE
# AND (player_matches.account_id = 183378746 OR player_matches.account_id = 207829314 OR player_matches.account_id = 129958758 OR player_matches.account_id = 111620041 OR player_matches.account_id = 148215639 OR player_matches.account_id = 106305042 OR player_matches.account_id = 256156323 OR player_matches.account_id = 113331514 OR player_matches.account_id = 321580662 OR player_matches.account_id = 302214028 OR player_matches.account_id = 150961567 OR player_matches.account_id = 173978074 OR player_matches.account_id = 147767183 OR player_matches.account_id = 111114687 OR player_matches.account_id = 320252024 OR player_matches.account_id = 126212866 OR player_matches.account_id = 154715080 OR player_matches.account_id = 94155156 OR player_matches.account_id = 25907144 OR player_matches.account_id = 86745912 OR player_matches.account_id = 56351509 OR player_matches.account_id = 117483894 OR player_matches.account_id = 90125566 OR player_matches.account_id = 167976729 OR player_matches.account_id = 401792574 OR player_matches.account_id = 93618577)
# AND matches.start_time >= extract(epoch from timestamp '2023-04-21T07:00:00.000Z')
# ORDER BY matches.match_id NULLS LAST
# LIMIT 2000
match_id_api2='https://api.opendota.com/api/explorer?sql=SELECT%0Amatches.match_id%2C%0Amatches.start_time%2C%0A((player_matches.player_slot%20%3C%20128)%20%3D%20matches.radiant_win)%20win%2C%0Aplayer_matches.hero_id%2C%0Aplayer_matches.account_id%2C%0Aleagues.name%20leaguename%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0AJOIN%20leagues%20using(leagueid)%0AJOIN%20player_matches%20using(match_id)%0AJOIN%20heroes%20on%20heroes.id%20%3D%20player_matches.hero_id%0ALEFT%20JOIN%20notable_players%20ON%20notable_players.account_id%20%3D%20player_matches.account_id%0ALEFT%20JOIN%20teams%20using(team_id)%0AWHERE%20TRUE%0AAND%20(player_matches.account_id%20%3D%20183378746%20OR%20player_matches.account_id%20%3D%20207829314%20OR%20player_matches.account_id%20%3D%20129958758%20OR%20player_matches.account_id%20%3D%20111620041%20OR%20player_matches.account_id%20%3D%20148215639%20OR%20player_matches.account_id%20%3D%20106305042%20OR%20player_matches.account_id%20%3D%20256156323%20OR%20player_matches.account_id%20%3D%20113331514%20OR%20player_matches.account_id%20%3D%20321580662%20OR%20player_matches.account_id%20%3D%20302214028%20OR%20player_matches.account_id%20%3D%20150961567%20OR%20player_matches.account_id%20%3D%20173978074%20OR%20player_matches.account_id%20%3D%20147767183%20OR%20player_matches.account_id%20%3D%20111114687%20OR%20player_matches.account_id%20%3D%20320252024%20OR%20player_matches.account_id%20%3D%20126212866%20OR%20player_matches.account_id%20%3D%20154715080%20OR%20player_matches.account_id%20%3D%2094155156%20OR%20player_matches.account_id%20%3D%2025907144%20OR%20player_matches.account_id%20%3D%2086745912%20OR%20player_matches.account_id%20%3D%2056351509%20OR%20player_matches.account_id%20%3D%20117483894%20OR%20player_matches.account_id%20%3D%2090125566%20OR%20player_matches.account_id%20%3D%20167976729%20OR%20player_matches.account_id%20%3D%20401792574%20OR%20player_matches.account_id%20%3D%2093618577)%0AAND%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272023-04-21T07%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.match_id%20NULLS%20LAST%0ALIMIT%202000'

In [49]:
# 11-15
# SELECT
        
# matches.match_id,
# matches.start_time,
# ((player_matches.player_slot < 128) = matches.radiant_win) win,
# player_matches.hero_id,
# player_matches.account_id,
# leagues.name leaguename
# FROM matches
# JOIN match_patch using(match_id)
# JOIN leagues using(leagueid)
# JOIN player_matches using(match_id)
# JOIN heroes on heroes.id = player_matches.hero_id
# LEFT JOIN notable_players ON notable_players.account_id = player_matches.account_id
# LEFT JOIN teams using(team_id)
# WHERE TRUE
# AND (player_matches.account_id = 85696663 OR player_matches.account_id = 383867949 OR player_matches.account_id = 155162307 OR player_matches.account_id = 308163407 OR player_matches.account_id = 186837494 OR player_matches.account_id = 164532005 OR player_matches.account_id = 9403474 OR player_matches.account_id = 190826739 OR player_matches.account_id = 127617979 OR player_matches.account_id = 87278757 OR player_matches.account_id = 121404228 OR player_matches.account_id = 355168766 OR player_matches.account_id = 94296097 OR player_matches.account_id = 324277900 OR player_matches.account_id = 156328257 OR player_matches.account_id = 139876032 OR player_matches.account_id = 129430590 OR player_matches.account_id = 138857296 OR player_matches.account_id = 143693439 OR player_matches.account_id = 137193239 OR player_matches.account_id = 101259972 OR player_matches.account_id = 375507918 OR player_matches.account_id = 100471531 OR player_matches.account_id = 140153524 OR player_matches.account_id = 301750126)
# AND matches.start_time >= extract(epoch from timestamp '2023-04-21T07:00:00.000Z')
# ORDER BY matches.match_id NULLS LAST
# LIMIT 2000

match_id_api3='https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%2C%0A((player_matches.player_slot%20%3C%20128)%20%3D%20matches.radiant_win)%20win%2C%0Aplayer_matches.hero_id%2C%0Aplayer_matches.account_id%2C%0Aleagues.name%20leaguename%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0AJOIN%20leagues%20using(leagueid)%0AJOIN%20player_matches%20using(match_id)%0AJOIN%20heroes%20on%20heroes.id%20%3D%20player_matches.hero_id%0ALEFT%20JOIN%20notable_players%20ON%20notable_players.account_id%20%3D%20player_matches.account_id%0ALEFT%20JOIN%20teams%20using(team_id)%0AWHERE%20TRUE%0AAND%20(player_matches.account_id%20%3D%2085696663%20OR%20player_matches.account_id%20%3D%20383867949%20OR%20player_matches.account_id%20%3D%20155162307%20OR%20player_matches.account_id%20%3D%20308163407%20OR%20player_matches.account_id%20%3D%20186837494%20OR%20player_matches.account_id%20%3D%20164532005%20OR%20player_matches.account_id%20%3D%209403474%20OR%20player_matches.account_id%20%3D%20190826739%20OR%20player_matches.account_id%20%3D%20127617979%20OR%20player_matches.account_id%20%3D%2087278757%20OR%20player_matches.account_id%20%3D%20121404228%20OR%20player_matches.account_id%20%3D%20355168766%20OR%20player_matches.account_id%20%3D%2094296097%20OR%20player_matches.account_id%20%3D%20324277900%20OR%20player_matches.account_id%20%3D%20156328257%20OR%20player_matches.account_id%20%3D%20139876032%20OR%20player_matches.account_id%20%3D%20129430590%20OR%20player_matches.account_id%20%3D%20138857296%20OR%20player_matches.account_id%20%3D%20143693439%20OR%20player_matches.account_id%20%3D%20137193239%20OR%20player_matches.account_id%20%3D%20101259972%20OR%20player_matches.account_id%20%3D%20375507918%20OR%20player_matches.account_id%20%3D%20100471531%20OR%20player_matches.account_id%20%3D%20140153524%20OR%20player_matches.account_id%20%3D%20301750126)%0AAND%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272023-04-21T07%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.match_id%20NULLS%20LAST%0ALIMIT%202000'

In [53]:
# 15-20
# SELECT
        
# matches.match_id,
# matches.start_time,
# ((player_matches.player_slot < 128) = matches.radiant_win) win,
# player_matches.hero_id,
# player_matches.account_id,
# leagues.name leaguename
# FROM matches
# JOIN match_patch using(match_id)
# JOIN leagues using(leagueid)
# JOIN player_matches using(match_id)
# JOIN heroes on heroes.id = player_matches.hero_id
# LEFT JOIN notable_players ON notable_players.account_id = player_matches.account_id
# LEFT JOIN teams using(team_id)
# WHERE TRUE
# AND (player_matches.account_id = 171262902 OR player_matches.account_id = 96803083 OR player_matches.account_id = 196878136 OR player_matches.account_id = 127565532 OR player_matches.account_id = 140288368 OR player_matches.account_id = 148180390 OR player_matches.account_id = 151069590 OR player_matches.account_id = 12231202 OR player_matches.account_id = 153125655 OR player_matches.account_id = 106755427 OR player_matches.account_id = 103910993 OR player_matches.account_id = 35504297 OR player_matches.account_id = 352545711 OR player_matches.account_id = 408729743 OR player_matches.account_id = 164685175 OR player_matches.account_id = 119631156 OR player_matches.account_id = 157989498 OR player_matches.account_id = 399862798 OR player_matches.account_id = 337575662 OR player_matches.account_id = 156029808 OR player_matches.account_id = 389022189 OR player_matches.account_id = 173378701 OR player_matches.account_id = 331855530 OR player_matches.account_id = 431770905 OR player_matches.account_id = 480412663 OR player_matches.account_id = 317880638 OR player_matches.account_id = 124801257 OR player_matches.account_id = 5150808)
# AND matches.start_time >= extract(epoch from timestamp '2023-04-21T07:00:00.000Z')
# ORDER BY matches.match_id NULLS LAST
# LIMIT 2000
match_id_api4='https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%2C%0A((player_matches.player_slot%20%3C%20128)%20%3D%20matches.radiant_win)%20win%2C%0Aplayer_matches.hero_id%2C%0Aplayer_matches.account_id%2C%0Aleagues.name%20leaguename%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0AJOIN%20leagues%20using(leagueid)%0AJOIN%20player_matches%20using(match_id)%0AJOIN%20heroes%20on%20heroes.id%20%3D%20player_matches.hero_id%0ALEFT%20JOIN%20notable_players%20ON%20notable_players.account_id%20%3D%20player_matches.account_id%0ALEFT%20JOIN%20teams%20using(team_id)%0AWHERE%20TRUE%0AAND%20(player_matches.account_id%20%3D%20171262902%20OR%20player_matches.account_id%20%3D%2096803083%20OR%20player_matches.account_id%20%3D%20196878136%20OR%20player_matches.account_id%20%3D%20127565532%20OR%20player_matches.account_id%20%3D%20140288368%20OR%20player_matches.account_id%20%3D%20148180390%20OR%20player_matches.account_id%20%3D%20151069590%20OR%20player_matches.account_id%20%3D%2012231202%20OR%20player_matches.account_id%20%3D%20153125655%20OR%20player_matches.account_id%20%3D%20106755427%20OR%20player_matches.account_id%20%3D%20103910993%20OR%20player_matches.account_id%20%3D%2035504297%20OR%20player_matches.account_id%20%3D%20352545711%20OR%20player_matches.account_id%20%3D%20408729743%20OR%20player_matches.account_id%20%3D%20164685175%20OR%20player_matches.account_id%20%3D%20119631156%20OR%20player_matches.account_id%20%3D%20157989498%20OR%20player_matches.account_id%20%3D%20399862798%20OR%20player_matches.account_id%20%3D%20337575662%20OR%20player_matches.account_id%20%3D%20156029808%20OR%20player_matches.account_id%20%3D%20389022189%20OR%20player_matches.account_id%20%3D%20173378701%20OR%20player_matches.account_id%20%3D%20331855530%20OR%20player_matches.account_id%20%3D%20431770905%20OR%20player_matches.account_id%20%3D%20480412663%20OR%20player_matches.account_id%20%3D%20317880638%20OR%20player_matches.account_id%20%3D%20124801257%20OR%20player_matches.account_id%20%3D%205150808)%0AAND%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272023-04-21T07%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.match_id%20NULLS%20LAST%0ALIMIT%202000'


In [56]:
# 20-25

# SELECT
        
# matches.match_id,
# matches.start_time,
# ((player_matches.player_slot < 128) = matches.radiant_win) win,
# player_matches.hero_id,
# player_matches.account_id,
# leagues.name leaguename
# FROM matches
# JOIN match_patch using(match_id)
# JOIN leagues using(leagueid)
# JOIN player_matches using(match_id)
# JOIN heroes on heroes.id = player_matches.hero_id
# LEFT JOIN notable_players ON notable_players.account_id = player_matches.account_id
# LEFT JOIN teams using(team_id)
# WHERE TRUE
# AND (player_matches.account_id = 228517469 OR player_matches.account_id = 169756712 OR player_matches.account_id = 218367241 OR player_matches.account_id = 136737280 OR player_matches.account_id = 145331311 OR player_matches.account_id = 110759067 OR player_matches.account_id = 152859296 OR player_matches.account_id = 150862738 OR player_matches.account_id = 181716137 OR player_matches.account_id = 118559150 OR player_matches.account_id = 153727297 OR player_matches.account_id = 96183976 OR player_matches.account_id = 320219866 OR player_matches.account_id = 140297552 OR player_matches.account_id = 152168157 OR player_matches.account_id = 87197791 OR player_matches.account_id = 105248644 OR player_matches.account_id = 82262664 OR player_matches.account_id = 34505203 OR player_matches.account_id = 101356886)
# AND matches.start_time >= extract(epoch from timestamp '2023-04-21T07:00:00.000Z')
# ORDER BY matches.match_id NULLS LAST
# LIMIT 200
match_id_api5='https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%2C%0A((player_matches.player_slot%20%3C%20128)%20%3D%20matches.radiant_win)%20win%2C%0Aplayer_matches.hero_id%2C%0Aplayer_matches.account_id%2C%0Aleagues.name%20leaguename%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0AJOIN%20leagues%20using(leagueid)%0AJOIN%20player_matches%20using(match_id)%0AJOIN%20heroes%20on%20heroes.id%20%3D%20player_matches.hero_id%0ALEFT%20JOIN%20notable_players%20ON%20notable_players.account_id%20%3D%20player_matches.account_id%0ALEFT%20JOIN%20teams%20using(team_id)%0AWHERE%20TRUE%0AAND%20(player_matches.account_id%20%3D%20228517469%20OR%20player_matches.account_id%20%3D%20169756712%20OR%20player_matches.account_id%20%3D%20218367241%20OR%20player_matches.account_id%20%3D%20136737280%20OR%20player_matches.account_id%20%3D%20145331311%20OR%20player_matches.account_id%20%3D%20110759067%20OR%20player_matches.account_id%20%3D%20152859296%20OR%20player_matches.account_id%20%3D%20150862738%20OR%20player_matches.account_id%20%3D%20181716137%20OR%20player_matches.account_id%20%3D%20118559150%20OR%20player_matches.account_id%20%3D%20153727297%20OR%20player_matches.account_id%20%3D%2096183976%20OR%20player_matches.account_id%20%3D%20320219866%20OR%20player_matches.account_id%20%3D%20140297552%20OR%20player_matches.account_id%20%3D%20152168157%20OR%20player_matches.account_id%20%3D%2087197791%20OR%20player_matches.account_id%20%3D%20105248644%20OR%20player_matches.account_id%20%3D%2082262664%20OR%20player_matches.account_id%20%3D%2034505203%20OR%20player_matches.account_id%20%3D%20101356886)%0AAND%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272023-04-21T07%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.match_id%20NULLS%20LAST%0ALIMIT%20200'

# SELECT
        
# matches.match_id,
# matches.start_time,
# ((player_matches.player_slot < 128) = matches.radiant_win) win,
# player_matches.hero_id,
# player_matches.account_id,
# leagues.name leaguename
# FROM matches
# JOIN match_patch using(match_id)
# JOIN leagues using(leagueid)
# JOIN player_matches using(match_id)
# JOIN heroes on heroes.id = player_matches.hero_id
# LEFT JOIN notable_players ON notable_players.account_id = player_matches.account_id
# LEFT JOIN teams using(team_id)
# WHERE TRUE
# AND (player_matches.account_id = 136829091 OR player_matches.account_id = 184620877 OR player_matches.account_id = 346412363 OR player_matches.account_id = 38628747 OR player_matches.account_id = 97658618 OR player_matches.account_id = 238239590 OR player_matches.account_id = 87063175 OR player_matches.account_id = 81475303 OR player_matches.account_id = 50828662 OR player_matches.account_id = 110448679 OR player_matches.account_id = 84429681 OR player_matches.account_id = 126238768 OR player_matches.account_id = 134711350 OR player_matches.account_id = 147488585 OR player_matches.account_id = 158847773 OR player_matches.account_id = 381714118 OR player_matches.account_id = 189666416 OR player_matches.account_id = 185908355 OR player_matches.account_id = 138543123 OR player_matches.account_id = 1171877280 OR player_matches.account_id = 136421659 OR player_matches.account_id = 199222238 OR player_matches.account_id = 202217968 OR player_matches.account_id = 165671428 OR player_matches.account_id = 172099728 OR player_matches.account_id = 94049589 OR player_matches.account_id = 145065875 OR player_matches.account_id = 123023873 OR player_matches.account_id = 195108598)
# AND matches.start_time >= extract(epoch from timestamp '2023-04-21T07:00:00.000Z')
# ORDER BY matches.match_id NULLS LAST
# LIMIT 2000
match_id_api6='https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%2C%0A((player_matches.player_slot%20%3C%20128)%20%3D%20matches.radiant_win)%20win%2C%0Aplayer_matches.hero_id%2C%0Aplayer_matches.account_id%2C%0Aleagues.name%20leaguename%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0AJOIN%20leagues%20using(leagueid)%0AJOIN%20player_matches%20using(match_id)%0AJOIN%20heroes%20on%20heroes.id%20%3D%20player_matches.hero_id%0ALEFT%20JOIN%20notable_players%20ON%20notable_players.account_id%20%3D%20player_matches.account_id%0ALEFT%20JOIN%20teams%20using(team_id)%0AWHERE%20TRUE%0AAND%20(player_matches.account_id%20%3D%20136829091%20OR%20player_matches.account_id%20%3D%20184620877%20OR%20player_matches.account_id%20%3D%20346412363%20OR%20player_matches.account_id%20%3D%2038628747%20OR%20player_matches.account_id%20%3D%2097658618%20OR%20player_matches.account_id%20%3D%20238239590%20OR%20player_matches.account_id%20%3D%2087063175%20OR%20player_matches.account_id%20%3D%2081475303%20OR%20player_matches.account_id%20%3D%2050828662%20OR%20player_matches.account_id%20%3D%20110448679%20OR%20player_matches.account_id%20%3D%2084429681%20OR%20player_matches.account_id%20%3D%20126238768%20OR%20player_matches.account_id%20%3D%20134711350%20OR%20player_matches.account_id%20%3D%20147488585%20OR%20player_matches.account_id%20%3D%20158847773%20OR%20player_matches.account_id%20%3D%20381714118%20OR%20player_matches.account_id%20%3D%20189666416%20OR%20player_matches.account_id%20%3D%20185908355%20OR%20player_matches.account_id%20%3D%20138543123%20OR%20player_matches.account_id%20%3D%201171877280%20OR%20player_matches.account_id%20%3D%20136421659%20OR%20player_matches.account_id%20%3D%20199222238%20OR%20player_matches.account_id%20%3D%20202217968%20OR%20player_matches.account_id%20%3D%20165671428%20OR%20player_matches.account_id%20%3D%20172099728%20OR%20player_matches.account_id%20%3D%2094049589%20OR%20player_matches.account_id%20%3D%20145065875%20OR%20player_matches.account_id%20%3D%20123023873%20OR%20player_matches.account_id%20%3D%20195108598)%0AAND%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272023-04-21T07%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.match_id%20NULLS%20LAST%0ALIMIT%202000'


# SELECT
        
# matches.match_id,
# matches.start_time,
# ((player_matches.player_slot < 128) = matches.radiant_win) win,
# player_matches.hero_id,
# player_matches.account_id,
# leagues.name leaguename
# FROM matches
# JOIN match_patch using(match_id)
# JOIN leagues using(leagueid)
# JOIN player_matches using(match_id)
# JOIN heroes on heroes.id = player_matches.hero_id
# LEFT JOIN notable_players ON notable_players.account_id = player_matches.account_id
# LEFT JOIN teams using(team_id)
# WHERE TRUE
# AND (player_matches.account_id = 185059559 OR player_matches.account_id = 116249155 OR player_matches.account_id = 293731272 OR player_matches.account_id = 164199202 OR player_matches.account_id = 164962869 OR player_matches.account_id = 92028391 OR player_matches.account_id = 217472313 OR player_matches.account_id = 105045291 OR player_matches.account_id = 208181197 OR player_matches.account_id = 138880576 OR player_matches.account_id = 86953414 OR player_matches.account_id = 105344475 OR player_matches.account_id = 397250058 OR player_matches.account_id = 425588742 OR player_matches.account_id = 131246802 OR player_matches.account_id = 118948666 OR player_matches.account_id = 103850210 OR player_matches.account_id = 122049498 OR player_matches.account_id = 193884241 OR player_matches.account_id = 113435203 OR player_matches.account_id = 412753955 OR player_matches.account_id = 18180970 OR player_matches.account_id = 41231571 OR player_matches.account_id = 1014295667 OR player_matches.account_id = 125445069 OR player_matches.account_id = 317814449 OR player_matches.account_id = 898455820 OR player_matches.account_id = 107023378 OR player_matches.account_id = 92487440 OR player_matches.account_id = 241884166 OR player_matches.account_id = 296226696 OR player_matches.account_id = 131303632 OR player_matches.account_id = 107579895 OR player_matches.account_id = 86822085 OR player_matches.account_id = 81306398 OR player_matches.account_id = 84853828 OR player_matches.account_id = 184138153 OR player_matches.account_id = 206379758 OR player_matches.account_id = 119576842 OR player_matches.account_id = 116585378)
# AND matches.start_time >= extract(epoch from timestamp '2023-04-21T07:00:00.000Z')
# ORDER BY matches.match_id NULLS LAST
# LIMIT 2000

match_id_api7 = 'https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%2C%0A((player_matches.player_slot%20%3C%20128)%20%3D%20matches.radiant_win)%20win%2C%0Aplayer_matches.hero_id%2C%0Aplayer_matches.account_id%2C%0Aleagues.name%20leaguename%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0AJOIN%20leagues%20using(leagueid)%0AJOIN%20player_matches%20using(match_id)%0AJOIN%20heroes%20on%20heroes.id%20%3D%20player_matches.hero_id%0ALEFT%20JOIN%20notable_players%20ON%20notable_players.account_id%20%3D%20player_matches.account_id%0ALEFT%20JOIN%20teams%20using(team_id)%0AWHERE%20TRUE%0AAND%20(player_matches.account_id%20%3D%20185059559%20OR%20player_matches.account_id%20%3D%20116249155%20OR%20player_matches.account_id%20%3D%20293731272%20OR%20player_matches.account_id%20%3D%20164199202%20OR%20player_matches.account_id%20%3D%20164962869%20OR%20player_matches.account_id%20%3D%2092028391%20OR%20player_matches.account_id%20%3D%20217472313%20OR%20player_matches.account_id%20%3D%20105045291%20OR%20player_matches.account_id%20%3D%20208181197%20OR%20player_matches.account_id%20%3D%20138880576%20OR%20player_matches.account_id%20%3D%2086953414%20OR%20player_matches.account_id%20%3D%20105344475%20OR%20player_matches.account_id%20%3D%20397250058%20OR%20player_matches.account_id%20%3D%20425588742%20OR%20player_matches.account_id%20%3D%20131246802%20OR%20player_matches.account_id%20%3D%20118948666%20OR%20player_matches.account_id%20%3D%20103850210%20OR%20player_matches.account_id%20%3D%20122049498%20OR%20player_matches.account_id%20%3D%20193884241%20OR%20player_matches.account_id%20%3D%20113435203%20OR%20player_matches.account_id%20%3D%20412753955%20OR%20player_matches.account_id%20%3D%2018180970%20OR%20player_matches.account_id%20%3D%2041231571%20OR%20player_matches.account_id%20%3D%201014295667%20OR%20player_matches.account_id%20%3D%20125445069%20OR%20player_matches.account_id%20%3D%20317814449%20OR%20player_matches.account_id%20%3D%20898455820%20OR%20player_matches.account_id%20%3D%20107023378%20OR%20player_matches.account_id%20%3D%2092487440%20OR%20player_matches.account_id%20%3D%20241884166%20OR%20player_matches.account_id%20%3D%20296226696%20OR%20player_matches.account_id%20%3D%20131303632%20OR%20player_matches.account_id%20%3D%20107579895%20OR%20player_matches.account_id%20%3D%2086822085%20OR%20player_matches.account_id%20%3D%2081306398%20OR%20player_matches.account_id%20%3D%2084853828%20OR%20player_matches.account_id%20%3D%20184138153%20OR%20player_matches.account_id%20%3D%20206379758%20OR%20player_matches.account_id%20%3D%20119576842%20OR%20player_matches.account_id%20%3D%20116585378)%0AAND%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272023-04-21T07%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.match_id%20NULLS%20LAST%0ALIMIT%202000'

# SELECT
        
# matches.match_id,
# matches.start_time,
# ((player_matches.player_slot < 128) = matches.radiant_win) win,
# player_matches.hero_id,
# player_matches.account_id,
# leagues.name leaguename
# FROM matches
# JOIN match_patch using(match_id)
# JOIN leagues using(leagueid)
# JOIN player_matches using(match_id)
# JOIN heroes on heroes.id = player_matches.hero_id
# LEFT JOIN notable_players ON notable_players.account_id = player_matches.account_id
# LEFT JOIN teams using(team_id)
# WHERE TRUE
# AND (player_matches.account_id = 349495318 OR player_matches.account_id = 458287006 OR player_matches.account_id = 392169957 OR player_matches.account_id = 196400041 OR player_matches.account_id = 274078636 OR player_matches.account_id = 260408033 OR player_matches.account_id = 175825387 OR player_matches.account_id = 113051039 OR player_matches.account_id = 1102532597 OR player_matches.account_id = 324045303 OR player_matches.account_id = 1169684404 OR player_matches.account_id = 156662698 OR player_matches.account_id = 107081378 OR player_matches.account_id = 315272623 OR player_matches.account_id = 137129583 OR player_matches.account_id = 114645455 OR player_matches.account_id = 329589381 OR player_matches.account_id = 143786911 OR player_matches.account_id = 407321629 OR player_matches.account_id = 59463394 OR player_matches.account_id = 100758751 OR player_matches.account_id = 427017829 OR player_matches.account_id = 399920568 OR player_matches.account_id = 392385140 OR player_matches.account_id = 202464276 OR player_matches.account_id = 302568414 OR player_matches.account_id = 148480336)
# AND matches.start_time >= extract(epoch from timestamp '2023-04-21T07:00:00.000Z')
# ORDER BY matches.match_id NULLS LAST
# LIMIT 2000
match_id_api7='https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%2C%0A((player_matches.player_slot%20%3C%20128)%20%3D%20matches.radiant_win)%20win%2C%0Aplayer_matches.hero_id%2C%0Aplayer_matches.account_id%2C%0Aleagues.name%20leaguename%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0AJOIN%20leagues%20using(leagueid)%0AJOIN%20player_matches%20using(match_id)%0AJOIN%20heroes%20on%20heroes.id%20%3D%20player_matches.hero_id%0ALEFT%20JOIN%20notable_players%20ON%20notable_players.account_id%20%3D%20player_matches.account_id%0ALEFT%20JOIN%20teams%20using(team_id)%0AWHERE%20TRUE%0AAND%20(player_matches.account_id%20%3D%20349495318%20OR%20player_matches.account_id%20%3D%20458287006%20OR%20player_matches.account_id%20%3D%20392169957%20OR%20player_matches.account_id%20%3D%20196400041%20OR%20player_matches.account_id%20%3D%20274078636%20OR%20player_matches.account_id%20%3D%20260408033%20OR%20player_matches.account_id%20%3D%20175825387%20OR%20player_matches.account_id%20%3D%20113051039%20OR%20player_matches.account_id%20%3D%201102532597%20OR%20player_matches.account_id%20%3D%20324045303%20OR%20player_matches.account_id%20%3D%201169684404%20OR%20player_matches.account_id%20%3D%20156662698%20OR%20player_matches.account_id%20%3D%20107081378%20OR%20player_matches.account_id%20%3D%20315272623%20OR%20player_matches.account_id%20%3D%20137129583%20OR%20player_matches.account_id%20%3D%20114645455%20OR%20player_matches.account_id%20%3D%20329589381%20OR%20player_matches.account_id%20%3D%20143786911%20OR%20player_matches.account_id%20%3D%20407321629%20OR%20player_matches.account_id%20%3D%2059463394%20OR%20player_matches.account_id%20%3D%20100758751%20OR%20player_matches.account_id%20%3D%20427017829%20OR%20player_matches.account_id%20%3D%20399920568%20OR%20player_matches.account_id%20%3D%20392385140%20OR%20player_matches.account_id%20%3D%20202464276%20OR%20player_matches.account_id%20%3D%20302568414%20OR%20player_matches.account_id%20%3D%20148480336)%0AAND%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272023-04-21T07%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.match_id%20NULLS%20LAST%0ALIMIT%202000'

In [72]:
# SELECT
        
# matches.match_id,
# matches.start_time

# FROM matches
# JOIN match_patch using(match_id)

# WHERE matches.start_time >= extract(epoch from timestamp '2023-05-09T07:00:00.000Z')
# AND matches.start_time <= extract(epoch from timestamp '2023-05-14T07:00:00.000Z')
# ORDER BY matches.start_time DESC
# LIMIT 100000

match_id_latest='https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%0A%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0A%0AWHERE%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272023-05-09T07%3A00%3A00.000Z%27)%0AAND%20matches.start_time%20%3C%3D%20extract(epoch%20from%20timestamp%20%272023-05-14T07%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.start_time%20DESC%0ALIMIT%20100000'

match_data = requests.get(match_id_latest).json()
match_ids = [row['match_id'] for row in match_data['rows']]

# match_data = requests.get(match_id_api2).json()
# match_ids.extend([row['match_id'] for row in match_data['rows']])

# match_data = requests.get(match_id_api3).json()
# match_ids.extend([row['match_id'] for row in match_data['rows']])

# match_data = requests.get(match_id_api4).json()
# match_ids.extend([row['match_id'] for row in match_data['rows']])

# match_data = requests.get(match_id_api5).json()
# match_ids.extend([row['match_id'] for row in match_data['rows']])

# match_data = requests.get(match_id_api6).json()
# match_ids.extend([row['match_id'] for row in match_data['rows']])

# match_data = requests.get(match_id_api7).json()
# match_ids.extend([row['match_id'] for row in match_data['rows']])

print(f"Altogether we have {len(set(match_ids))} matches.")

Altogether we have 303 matches.


In [86]:
api_new_version='https://api.opendota.com/api/explorer?sql=SELECT%0A%20%20%20%20%20%20%20%20%0Amatches.match_id%2C%0Amatches.start_time%0A%0AFROM%20matches%0AJOIN%20match_patch%20using(match_id)%0A%0AWHERE%20matches.start_time%20%3E%3D%20extract(epoch%20from%20timestamp%20%272023-04-21T07%3A00%3A00.000Z%27)%0AAND%20matches.start_time%20%3C%3D%20extract(epoch%20from%20timestamp%20%272023-05-13T07%3A00%3A00.000Z%27)%0AORDER%20BY%20matches.start_time%20DESC%0ALIMIT%20100000'
match_data = requests.get(api_new_version).json()
match_ids = [row['match_id'] for row in match_data['rows']]
print(f"Altogether we have {len(set(match_ids))} matches.")

Altogether we have 1720 matches.


In [88]:
base_id = match_ids[0]
base_id

7152103586

In [69]:
attributes_list= ['region',
 'isRadiant',
 'win',
 'lose',
 'total_gold',
 'total_xp',
 'kills_per_min',
 'kda',
 'abandons',
 'neutral_kills',
 'tower_kills',
 'courier_kills',
 'lane_kills',
 'hero_kills',
 'observer_kills',
 'sentry_kills',
 'roshan_kills',
 'necronomicon_kills',
 'ancient_kills',
 'buyback_count',
 'observer_uses',
 'sentry_uses']

In [122]:
from datetime import datetime
import time
# https://www.opendota.com/api-keys
def get_hero_position_and_role(row_dict_list, player_dict_list, match_id, matches_list):
    # Please add ?api_key=YOUR_API after {match_id} to accelerate the api call
    # Refer the opendota website https://www.opendota.com/api-keys
    match_json = get_api_json(f"https://api.opendota.com/api/matches/{match_id}", loop=True)
    matches_list.append(match_json)
    
    # match_id = match_json['match_id']
    # I have tested the following attributes and generally there will be no error
    row_dicts = []
    win_camp = None
    start_time, duration, game_mode = None, None, None
    radiant_score, dire_score = None, None
    try:
        if match_json["radiant_win"]==True:
            win_camp = 'radiant'
        else: 
            win_camp = 'dire'
        if match_json['game_mode']:  # mode 1: all pick, mode 2: captain mode
            game_mode = match_json['game_mode']
        else:
            game_mode = None
        start_time = match_json["start_time"]
        start_time = datetime.fromtimestamp(start_time).strftime('%y-%m-%d')
        duration = match_json["duration"]
        radiant_score = match_json['radiant_score'] 
        dire_score = match_json['dire_score'] 
    except:
        print(f'unknown result id: {match_id}')

    if 'players' in match_json.keys():
        for p in match_json['players']:
            row_dict = {}
            row_dict['match_id'] = match_id
            row_dict['camp'] = 'radiant' if p['isRadiant'] else 'dire'
            row_dict['hero_id'] = p['hero_id']
            row_dict['hero_name'] = hero_id_to_name[p['hero_id']]
            if 'name' in p:
                row_dict['player'] = p['name']
            else:
                row_dict['player'] = None
            row_dict['player_id'] = p['account_id']
            if 'lane_role' in p:
                row_dict["lane_role"] = {1: "safe", 2: "mid", 3: "off", 4:'other'}[p["lane_role"]]
            else:
                row_dict["lane_role"] = None
            if p["lh_t"] and len(p["lh_t"])>=6:
                # print(p["lh_t"])
                row_dict["6_min_last_hits"] = p["lh_t"][6]
            else:
                row_dict["6_min_last_hits"] = None
            
            purchase_ward_observer = p["purchase_ward_observer"] if "purchase_ward_observer" in p else 0
            purchase_ward_sentry = p["purchase_ward_sentry"] if "purchase_ward_sentry" in p else 0
            row_dict["purchase_ward_count"] = purchase_ward_observer + purchase_ward_sentry
            row_dict['gold_per_min'] = p['gold_per_min']
            row_dict['hero_damage'] = p['hero_damage']
            row_dict['tower_damage'] = p['tower_damage']
            if 'rank_tier' in p:
                row_dict['rank_tier'] = p['rank_tier']
            for a in attributes_list:
                if a in p:
                    row_dict[a] = p[a]
                else:
                    row_dict[a] = None

            row_dicts.append(row_dict)
            player_dict_list.append(row_dict)

        w_df = pd.DataFrame(row_dicts)
        sorted_core_df = w_df.sort_values("6_min_last_hits", ascending=False).head(6)  # find the 6 heros with max last hits in 6 mins --> cores
        sorted_gpm_radiant_df = w_df[w_df['camp']=='radiant'].sort_values("gold_per_min", ascending=False)
        sorted_gpm_dire_df = w_df[w_df['camp']=='dire'].sort_values("gold_per_min", ascending=False)
        # sorted_ward_df = w_df.sort_values('purchase_ward_count', ascending=False).head(4)  # find 4 heros with max wards purchase --> support TAT
        def get_role(r):
            # define the cores first(cores have higher hits), define support then
            if r.lane_role=='safe':
                if r.hero_id in sorted_core_df.hero_id.tolist():
                    return 'carry'
                else:
                    return 'hard_support'
            elif r.lane_role=='off':
                if r.hero_id in sorted_core_df.hero_id.tolist():
                    return 'offlane'
                else:
                    return 'soft_support'
            elif r.lane_role=='mid':
                if r.hero_id in sorted_core_df.hero_id.tolist():
                    return 'mid'
                else:
                    return 'Unknown'
            else: # if no lane role
                if r.hero_id==sorted_gpm_radiant_df.hero_id.tolist()[0] or r.hero_id==sorted_gpm_dire_df.hero_id.tolist()[0]:
                    return 'carry'
                elif r.hero_id==sorted_gpm_radiant_df.hero_id.tolist()[1] or r.hero_id==sorted_gpm_dire_df.hero_id.tolist()[1]:
                    return 'mid'
                elif r.hero_id==sorted_gpm_radiant_df.hero_id.tolist()[2] or r.hero_id==sorted_gpm_dire_df.hero_id.tolist()[2]:
                    return 'offlane'
                elif r.hero_id==sorted_gpm_radiant_df.hero_id.tolist()[3] or r.hero_id==sorted_gpm_dire_df.hero_id.tolist()[3]:
                    return 'soft_support'
                elif r.hero_id==sorted_gpm_radiant_df.hero_id.tolist()[4] or r.hero_id==sorted_gpm_dire_df.hero_id.tolist()[4]:
                    return 'hard_support'
                else:
                    return 'Unknown'
            
            # elif r.lane_role=='safe' and r.hero_id in sorted_ward_df.hero_id.tolist():
            #     return 'soft_support'
            # elif r.hero_id in sorted_ward_df.hero_id.tolist():
            #     return 'hard_support'
        # print(sorted_ward_df)
        w_df['role'] = w_df.apply(lambda r:get_role(r), axis=1)

        radiant_df = w_df[w_df['camp']=='radiant']
        dire_df = w_df[w_df['camp']=='dire']

        hero_positions_radiant = {radiant_df.hero_name.tolist()[i]:[radiant_df.role.tolist()[i], radiant_df.player_id.tolist()[i]] for i in range(len(radiant_df))}
        hero_positions_dire = {dire_df.hero_name.tolist()[i]:[dire_df.role.tolist()[i], dire_df.player_id.tolist()[i]] for i in range(len(dire_df))}
        
        res_dicts = {}
        res_dicts['match_id'] = match_id
        res_dicts['start_time'] = start_time
        res_dicts['duration'] = duration
        res_dicts['game_mode'] = game_mode
        res_dicts['win_camp'] = win_camp
        res_dicts['radiant_hero'] = hero_positions_radiant
        res_dicts['dire_hero'] = hero_positions_dire
        res_dicts['radiant_score'] = radiant_score
        res_dicts['dire_score'] = dire_score
        

        row_dict_list.append(res_dicts)
    else:
        print(f"Problem id: {match_id}")
    print(f"current length: {len(row_dict_list)}")
    # print(f"current length: {count}")
    return row_dict_list, player_dict_list, matches_list

team_dict_list = []
player_dict_list = []
count = 1
matches_list = []
for i in range(10):
    # count += 1
    new_id = base_id+i
    if count % 60 == 0:
        time.sleep(65) # free tier 60 calls per minute
    count += 1
    try:
        if get_api_json(f"https://api.opendota.com/api/matches/{new_id}", loop=True):
            team_dict_list, player_dict_list, matches_list = get_hero_position_and_role(team_dict_list, player_dict_list, new_id, matches_list)
    except:
        print(count)
        # break


# with open("data.json", "rb") as f:
#     for match_json in ijson.items(f, "item"):
#         # if count % 1200 == 0:
#         #     time.sleep(65) # free tier 60 calls per minute
#         count += 1
#         if count==30000:
#             team_df = pd.DataFrame(team_dict_list)
#             team_df.to_csv(f'team{count}.csv', index=False, header=True)

#             player_df = pd.DataFrame(player_dict_list)
#             # player_df.head(10)
#             player_df.to_csv(f'player{count}.csv', index=False, header=True)
#             team_dict_list, player_dict_list, matches_list = [], [], []
#         team_dict_list, player_dict_list, matches_list = get_hero_position_and_role(team_dict_list, player_dict_list, match_json, matches_list)
        
        
# for match_id in match_ids:
#     # count += 1
#     if count % 1200 == 0:
#         time.sleep(65) # free tier 60 calls per minute
#     count += 1
#     team_dict_list, player_dict_list, matches_list = get_hero_position_and_role(team_dict_list, player_dict_list, match_id, matches_list)

current length: 1
unknown result id: 7152103587
Problem id: 7152103587
current length: 1
unknown result id: 7152103588
Problem id: 7152103588
current length: 1
current length: 2
current length: 3
current length: 4
unknown result id: 7152103592
Problem id: 7152103592
current length: 4
unknown result id: 7152103593
Problem id: 7152103593
current length: 4
current length: 5
current length: 6


In [130]:
matches_list[-1]

{'match_id': 7152103595,
 'barracks_status_dire': 51,
 'barracks_status_radiant': 63,
 'chat': None,
 'cluster': 144,
 'cosmetics': None,
 'dire_score': 37,
 'dire_team_id': None,
 'draft_timings': None,
 'duration': 2649,
 'engine': 1,
 'first_blood_time': 9,
 'game_mode': 22,
 'human_players': 10,
 'leagueid': 0,
 'lobby_type': 7,
 'match_seq_num': 6009706067,
 'negative_votes': 0,
 'objectives': None,
 'picks_bans': [{'is_pick': True, 'hero_id': 101, 'team': 0, 'order': 0},
  {'is_pick': True, 'hero_id': 84, 'team': 1, 'order': 1},
  {'is_pick': True, 'hero_id': 20, 'team': 1, 'order': 2},
  {'is_pick': True, 'hero_id': 45, 'team': 0, 'order': 3},
  {'is_pick': True, 'hero_id': 138, 'team': 0, 'order': 4},
  {'is_pick': True, 'hero_id': 26, 'team': 1, 'order': 5},
  {'is_pick': True, 'hero_id': 93, 'team': 1, 'order': 6},
  {'is_pick': True, 'hero_id': 104, 'team': 0, 'order': 7},
  {'is_pick': True, 'hero_id': 36, 'team': 0, 'order': 8},
  {'is_pick': True, 'hero_id': 40, 'team': 1

In [134]:
# len(match_ids)
for match_id in match_ids[:5]:
    match = get_api_json(f"https://api.opendota.com/api/matches/{match_id}", loop=True)
    print(len(match['chat']))
    

23
17
146
29
34


In [123]:
team_dict_list[-1]

# {'match_id': 7146139538,
#  'start_time': '23-05-08',
#  'duration': 1409,
#  'game_mode': 2,
#  'win_camp': 'dire',
#  'radiant_hero': {'Death Prophet': ['mid', 1459205302],
#   'Jakiro': ['hard_support', 1459031599],
#   'Underlord': ['offlane', 1423366323],
#   'Gyrocopter': ['soft_support', 1458610209],
#   'Naga Siren': ['carry', 1459124273]},
#  'dire_hero': {'Queen of Pain': ['mid', 1526386891],
#   'Disruptor': ['hard_support', 1513040880],
#   'Templar Assassin': ['carry', 1427100407],
#   'Pudge': ['soft_support', 1517400698],
#   'Mars': ['offlane', 1529282248]},
#  'radiant_score': 14,
#  'dire_score': 34}

{'match_id': 7152103595,
 'start_time': '23-05-12',
 'duration': 2649,
 'game_mode': 22,
 'win_camp': 'radiant',
 'radiant_hero': {'Skywrath Mage': ['hard_support', 1266567363.0],
  'Necrophos': ['carry', nan],
  'Muerta': ['mid', 104341393.0],
  'Pugna': ['soft_support', 191756796.0],
  'Legion Commander': ['offlane', nan]},
 'dire_hero': {'Lion': ['soft_support', nan],
  'Vengeful Spirit': ['hard_support', nan],
  'Slark': ['mid', 393153030.0],
  'Ogre Magi': ['carry', nan],
  'Venomancer': ['offlane', nan]},
 'radiant_score': 55,
 'dire_score': 37}

In [94]:
import json 

# Combine dictionaries into a list 
dict_list = matches_list
len(dict_list)
 
# Write list of dictionaries to JSON file (optional)
with open('data0513.json', 'w') as f: 
    json.dump(dict_list, f) 

team_df = pd.DataFrame(team_dict_list)
team_df.to_csv('team_0513.csv', index=False, header=True)

player_df = pd.DataFrame(player_dict_list)
# player_df.head(10)
player_df.to_csv('player_0513.csv', index=False, header=True)

In [105]:
match_ids[-1]

7116890950

In [104]:
# tmp = pd.read_csv('player_0513.csv')
team_df

,match_id,start_time,duration,game_mode,win_camp,radiant_hero,dire_hero,radiant_score,dire_score
0,7152103586,23-05-12,2230,2,dire,"{'Shadow Fiend': ['carry', 170558809], 'Nyx As...","{'Warlock': ['hard_support', 240899847], 'Medu...",42,47
1,7152103589,23-05-12,2141,22,dire,"{'Necrophos': [None, None], 'Grimstroke': [Non...","{'Rubick': [None, None], 'Shadow Fiend': [None...",32,42
2,7152103590,23-05-12,2360,22,dire,"{'Gyrocopter': [None, 255593943.0], 'Undying':...","{'Mars': [None, 77474220.0], 'Winter Wyvern': ...",27,46
3,7152103591,23-05-12,1655,23,radiant,"{'Venomancer': [None, 140760641.0], 'Dark Will...","{'Gyrocopter': [None, 157918300.0], 'Zeus': [N...",43,26
4,7152103594,23-05-12,2803,22,dire,"{'Terrorblade': [None, 81806434.0], 'Treant Pr...","{'Phantom Lancer': [None, nan], 'Rubick': [Non...",34,45
...,...,...,...,...,...,...,...,...,...
15854,7152168813,23-05-13,2422,22,dire,"{'Ursa': [None, nan], 'Slark': [None, nan], 'T...","{'Sand King': [None, nan], 'Pudge': [None, 242...",28,46
15855,7152168814,23-05-13,2800,3,radiant,"{'Gyrocopter': [None, 144581226.0], 'Bloodseek...","{'Invoker': [None, 174584545.0], 'Hoodwink': [...",76,32
15856,7152168832,23-05-13,2833,22,dire,"{'Venomancer': [None, 913658882.0], 'Tidehunte...","{'Tinker': [None, nan], 'Shadow Fiend': [None,...",42,62
15857,7152168833,23-05-13,2808,22,dire,"{'Outworld Destroyer': [None, 264232956.0], 'P...","{'Enigma': [None, 404415449.0], 'Omniknight': ...",42,47


In [82]:
player1 = pd.read_csv('player.csv')
# player2 = pd.read_csv('player_append.csv')

# player_df = pd.concat([player1, player2], axis=0)
# player_df.to_csv('player.csv', index=False, header=True)

team1 = pd.read_csv('team.csv')
# team2 = pd.read_csv('team_append.csv')
# team_df = pd.concat([team1, team2], axis=0)
# team_df.to_csv('team.csv', index=False, header=True)
# team_df

team1

,match_id,start_time,duration,game_mode,win_camp,radiant_hero,dire_hero,radiant_score,dire_score
0,7146139538,23-05-08,1409,2,dire,"{'Death Prophet': ['mid', 1459205302], 'Jakiro...","{'Queen of Pain': ['mid', 1526386891], 'Disrup...",14,34
1,7146130696,23-05-08,2061,2,dire,"{'Kunkka': ['mid', 1045494300], 'Alchemist': [...","{'Ember Spirit': ['mid', 1133046148], 'Wraith ...",27,33
2,7146125801,23-05-08,2239,2,radiant,"{'Dark Seer': ['offlane', 1079923091], 'Phanto...","{'Axe': ['offlane', 236214375], 'Venomancer': ...",38,29
3,7146122207,23-05-08,2601,2,dire,"{'Slardar': ['offlane', 394524089], 'Medusa': ...","{'Faceless Void': ['carry', 381629803], 'Pugna...",20,40
4,7146119443,23-05-08,2959,2,dire,"{'Medusa': ['carry', 285770518], 'Storm Spirit...","{'Underlord': ['offlane', 1176819430], 'Void S...",31,50
...,...,...,...,...,...,...,...,...,...
49358,7146185549,23-05-09,1331,2,dire,"{'Medusa': ['offlane', 236214375], 'Mirana': [...","{'Phantom Lancer': ['carry', 336161828], 'Keep...",15,36
49359,7146177711,23-05-09,1615,2,radiant,"{'Pugna': ['hard_support', 966354824], 'Storm ...","{'Ursa': ['carry', 285770518], 'Tiny': ['mid',...",36,14
49360,7146174139,23-05-09,1226,2,dire,"{'Doom': ['offlane', 169786155], 'Shadow Fiend...","{'Muerta': ['carry', 381629803], 'Techies': ['...",8,30
49361,7146167573,23-05-09,1845,2,radiant,"{'Windranger': ['offlane', 76082922], 'Alchemi...","{'Legion Commander': ['offlane', 1103355581], ...",22,20


#### Download hero attributes using requests and BeautifulSoup

In [41]:
import requests
from bs4 import BeautifulSoup
# reference: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

urls = ['https://dota2.fandom.com/wiki/Table_of_hero_attributes']

In [42]:
r = requests.get(urls[0])
html_doc = r.content
soup = BeautifulSoup(html_doc, 'html.parser')

In [43]:
attr_list = []

def get_attributes(url, attr_list):
    r = requests.get(url)
    html_doc = r.content
    soup = BeautifulSoup(html_doc, 'html.parser')

    # print(soup.prettify())
    for tr in soup.tbody.find_all('tr')[1:]:
        hero_name = tr.a.get('href').split('/')[-1]
        tds = tr.find_all('td')
        attr_dict = {}
        attr_dict['hero'] = hero_name
        attr_dict['base_strength'] = float(tds[2].string)
        attr_dict['add_strength'] = float(tds[3].string)
        attr_dict['lvl30_strength'] = float(tds[4].string)
        attr_dict['base_agility'] = float(tds[5].string)
        attr_dict['add_agility'] = float(tds[6].string)
        attr_dict['lvl30_agility'] = float(tds[7].string)
        attr_dict['base_intelligence'] = float(tds[8].string)
        attr_dict['add_intelligence'] = float(tds[9].string)
        attr_dict['lvl30_intelligence'] = float(tds[10].string)
        attr_list.append(attr_dict)

for url in urls:
    get_attributes(url, attr_list)

In [44]:
print(f"Altogether we have {len(attr_list)} heroes.")

Altogether we have 124 heroes.


In [45]:
attr_list[2]

{'hero': 'Ancient_Apparition',
 'base_strength': 20.0,
 'add_strength': 1.9,
 'lvl30_strength': 75.1,
 'base_agility': 20.0,
 'add_agility': 2.2,
 'lvl30_agility': 83.8,
 'base_intelligence': 23.0,
 'add_intelligence': 3.4,
 'lvl30_intelligence': 121.6}

In [46]:
attribute_df = pd.DataFrame(attr_list)
attribute_df.to_csv('hero_attributes.csv', index=False, header=True)
attribute_df

,hero,base_strength,add_strength,lvl30_strength,base_agility,add_agility,lvl30_agility,base_intelligence,add_intelligence,lvl30_intelligence
0,Abaddon,22.0,2.2,85.8,23.0,1.3,60.7,18.0,1.2,52.8
1,Alchemist,23.0,2.7,101.3,22.0,1.5,65.5,25.0,1.8,77.2
2,Ancient_Apparition,20.0,1.9,75.1,20.0,2.2,83.8,23.0,3.4,121.6
3,Anti-Mage,19.0,1.6,65.4,24.0,2.8,105.2,12.0,1.8,64.2
4,Arc_Warden,22.0,2.6,97.4,20.0,3.0,107.0,24.0,2.6,99.4
...,...,...,...,...,...,...,...,...,...,...
119,Windranger,18.0,2.4,87.6,17.0,1.4,57.6,18.0,2.4,87.6
120,Winter_Wyvern,22.0,2.2,85.8,16.0,1.5,59.5,26.0,2.8,107.2
121,Witch_Doctor,18.0,2.3,84.7,13.0,1.4,53.6,22.0,3.3,117.7
122,Wraith_King,22.0,3.0,109.0,16.0,1.7,65.3,18.0,1.6,64.4
